In [1]:
import spacy
import csv
import pyprind
from torchtext import data
class dataProcesstor():
    def __init__(self,src,des,n):
        self.src = src
        self.des = des
        self.n = n
        self.nlp =  spacy.load('en_core_web_sm', disable=['ner', 'parser', 'tagger'])  
        self.dics = {}
        
        CONTEXT = data.Field()
        ANSWER  = data.Field()
        QUESTION = data.Field()
       
        # define col: {[source data col name]:[your data col name],Field}
        fields = {'context':('Context',CONTEXT),'question':('Question',QUESTION),'supporting_facts':('Answer',ANSWER)}
        dataset = data.TabularDataset(path = src,format='json',fields=fields)
        dataset = dataset.examples[0]
        
        for i in range (0,len(dataset.Context)):
            for title,sentence in dataset.Context[i]:
                self.dics[title] = sentence
        self.go(dataset)
    def getAnswer(self,ans):
        res = ''
        for title, sent_id in ans:
            if title in self.dics:
                if sent_id < len(self.dics[title]):
                    res += self.dics[title][sent_id]
        return res
    
    def getContext(self,text2DimList):
        res = ''
        for paragragh in text2DimList:
            res += '<sep>'.join(paragragh[1])
        return  res  

    def go(self,dataset):
        if self.n is -1:
            self.n = len(dataset.Question)
            print('this data has'+str(self.n)+'datas')
        pbar = pyprind.ProgBar(self.n)
        with open(self.des,'w',encoding="utf-8",newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['passage','question', 'answer'])
            
            for i in range (0,self.n):
                c =  self.getContext(dataset.Context[i])
                q =  dataset.Question[i]
                a =  self.getAnswer(dataset.Answer[i])
                writer.writerow([c,q,a])
                pbar.update()
        print('write down')   

In [2]:
from torchtext import data
from torchtext.vocab import GloVe
import torch
import spacy
from torchtext.data import Iterator, BucketIterator

class getHotpotData():
    def __init__(self,args,trainPath,devPath,):
        self.nlp = spacy.load('en_core_web_sm')   
        self.trainpath= trainPath
        self.devpath= devPath
        
        self.PASSAGE  = data.Field(tokenize = self.tokenizer,lower=True)
        self.QUESTION = data.Field(tokenize = self.tokenizer,lower=True)
        self.CONTEXT = data.Field(tokenize = self.tokenizer,lower=True)
        
        fields = {'passage':('Passage', self.PASSAGE),'answer':('Answer', self.ANSWER),'question':('Question', self.QUESTION)}
        
        self.train = data.TabularDataset(path = self.trainpath,format='csv',fields=fields)
        self.dev = data.TabularDataset(path = self.devpath,format='csv',fields=fields)
        
        self.PASSAGE.build_vocab(self.train,self.dev, vectors=GloVe(name='6B', dim=300))  
        self.QUESTION.build_vocab(self.train) 
        self.ANSWER.build_vocab(self.train)
        
        self.train_iter = data.BucketIterator(dataset=self.train, batch_size=args.batch_size, shuffle=True, sort_within_batch=False, repeat=False,device = args.gpu)
        self.dev_iter = data.BucketIterator(dataset=self.dev, batch_size=args.batch_size, shuffle=True, sort_within_batch=False, repeat=False,device = args.gpu)
        self.calculate_block_size(args.batch_size)
        print('load hotpot data done')
        
    def tokenizer(self,text):
        return [str(token) for token in self.nlp(text)]
    
    def calculate_block_size(self, B):
        data_lengths = []
        for e in self.train.examples:
            data_lengths.append(len(e.Passage))

        mean = np.mean(data_lengths)
        std = np.std(data_lengths)

        self.block_size = int((2 * (std * ((2 * np.log(B)) ** (1/2)) + mean)) ** (1/3))
    


In [3]:
    
trainpath = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/smalltrain2sep.csv'
devpath = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/smalldev2sep.csv'

src = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/dev.json'
des = 'C:/Users/User/Documents/3.NLP/Dataset/HotpotQA/small/smalldev100sep.csv'
#x = dataProcesstor(src,des,100)